# Assignment 4 CS 5316 Natural Language Processing

In [10]:
# from google.colab import drive
# drive.mount('/content/drive')
!unzip "/content/drive/My Drive/Colab Notebooks/msr_paraphrase.zip"

Archive:  /content/drive/My Drive/Colab Notebooks/msr_paraphrase.zip
  inflating: msr_paraphrase_test.txt  
  inflating: msr_paraphrase_train.txt  


In [0]:
d = "300"
!cp "/content/drive/My Drive/Colab Notebooks/glove_6B/glove.6B.300d.txt" "glove.6B.300d.txt"

In [0]:
import numpy as np
from IPython.display import Image
# Get the interactive Tools for Matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
# plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.decomposition import TruncatedSVD
from sklearn.utils.extmath import randomized_svd
from nltk import ngrams
import pandas as pd
import seaborn as sn
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, RepeatVector,Flatten, TimeDistributed, Input,Bidirectional,LocallyConnected1D,Conv1D,GlobalAveragePooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.layers import Embedding, LSTM ,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers
# from tensorflow.keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import to_categorical
# import tensorflow.keras.utils.to_categorical as to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import math
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.metrics import classification_report

# Final Assingmnet
This is going to be the final assignment for deep learning. Here is a very good visual for what you will be doing with
<a href="https://ibb.co/mh9Ks0j">deep learning.</a> Lets get started.......

# TASK 1 Paraphrase Detection
For this task we will be using the [ Microsoft Research Paraphrase Corpus ](https://www.microsoft.com/en-us/download/details.aspx?id=52398). The corpus consist of sentence pairs with 1 or 0 labels which identify if the sentences are paraphrase or not respectively.
<br>
To perform this task we will be using recurrenct neural network for this task specifically the [LSTM](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). RNN can be architected in multiple ways. Some of the possible ways are as follows:
<img src="archetecturernn.PNG">
The box in the bottom is the input, followed by the hidden layer (as the middle box), and the box on top is the output layer. The one-to-one architecture is the typical neural network (<i>vanila/Feed Forward</i>) with a hidden layer between the input and the output layer. Example uses of the above archetecture are as follows:
<ul>
    <li>One-to-many: input is an image and outputs are image captions</li>
    <li>Many-to-one: input is a movie's review <i>multiple words in input</i> and output is sentiment associated with the review <i>(we will be using a similar archetecture for our purpose)</i></li>
    <li>Many-to-many: machine translation of a sentence in one language to a sentence in another language, POS tagging etc</li>
</ul>
<br>
For this task we will also be using pre-trained word embeddings specificallly <a href="https://nlp.stanford.edu/projects/glove/">(GloVe Embeddings)</a>. Please download the paraphrase <a href="https://www.microsoft.com/en-us/download/details.aspx?id=52398">dataset</a> and glove.6B.zip from <a href="https://nlp.stanford.edu/projects/glove/">here</a>.

For this task you are required to implement the following archetecture, please use [keras functional API](https://www.tensorflow.org/guide/keras/functional) :
<img src="paraphrase.png">
If <a href="https://ibb.co/RSSjRM0">this</a> is you reaction after seeing the model archetecture dont worry we'll explain.
The model works as follows, there will be two inputs layers, one for each sentence followed by <b>shared</b> embedding layer which feed thier outputs to the shared LSTM, <b>take the final hidden state output</b> of both LSTM's and concatenate them. Finally feed the concatenated vector to a softmax output layer for classification.
<br>
<i>(The reason for using one shared embedding and LSTM layer so that the model learns sentence representation for all sentence pairs(x,x') in the dataset. If we were using two seperate LSTMS for x and x' we would need to double the dataset by having both (x,x') and (x',x) pairs so that both LSTM's see the entire train data distribution)</i>
The purpose for each layer in the model is as follows:
<ul>
    <li>Input takes the input sequences and feeds it to the next(you will need to specify the maximum size of a sentence as a parameter of this layer)</li>
    <li>Embedding layer, this layer takes the sequence input then for each word in the sequence generates a fixed size vector <i>(word embedding)</i>, this layer can be trained from scratch or can be configured to use pretrained embeddings with or without fine tuning. </li>
    <li>LSTM process the embedding vector sequences and at each step generates a hidden state vector(h)and cell memory vector(C)(<i>see diagram</i>), the keras LSTM layer returns three outputs (1) All the hidden states,(2) The final hidden state and (3) The final cell memory state<img src="lstm.png"></li>
    <li>The concatenation layer combines multple vectors into a single vector</li>
    <li>Finally the output layer predicts if sentence pairs were paraphrase or not</li>
</ul>
<b>Please refer to the TF-keras documentation for all the layers <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers">here</a></b>
<br><br><br>
Now that you understand the theoritical foundation for our approach lets move onto practical implementation.<br>
<h3>Data Preperation</h3>

<ul>
    <li> First we need to preprocess the data, convert the data to lower casing. Any other preprocessing procedures are optional but keep in mind that this will affect the performance of your model.</li>
    <li> To make training faster we will fix the maximum sequence length to 20 truncate the longer sequences.</li>
    <li> Split the data into test, train and validation in the ratio 20,70,10. Use <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">scikit_test_train_split</a> <br><i><b>Hint:</b> use the splitter twice to get desired data splits.</i></li>
    <li> Next we need the vocabulary, vocabulary size and to convert sentences to numeric sequences by representing each word with a numeric value which will make our implementation easier later on, use <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer">Tokenizer</a> from keras. <br><i>(Fit the tokenizer on train data and use the same tokenizer to convert train,test and validation data to numeric sequences)</i> </li>
    <li>  Use <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences"> pad sequences</a> to add post padding to all sentences that are shorter than maximum sequence length
        <i>(<b>extra info</b>: fit_on_text reserves value/index 0 for padding and assigns numeric value to words starting from index 1)</i></li>
</ul>
<h3>Loading embeddings</h3>
<ul>
    <li> To use pretrained embeddings in tf keras embedding layer requires a dictionary, we need to create a dictionary whose keys will be numeric word representations and values will be the embedding vectors.</li>
    <li> First step is to load the word embedding pairs from the glove file into a dictionary.</li>
    <li> Next we will create a dictionary for our dataset's vocabulary. Copy all the word embeddings for words that are in our vocabulary and in the glove dictionary, if a word exists in our vocabulary but does not exist in glove dictionary create a zero vector of embedding dimension size and add it to the dictionary.</li>
</ul>
<h3>Create Model</h3>
<ul>
    <li> Create the model using <a href"https://www.tensorflow.org/guide/keras/functional">functional API</a></li>
    <li> Hints: The emebedding layer has a parameter that allows you to use pretrained embeddings, for shared layers read the section of shared layer weights in function API docs</li>

In [0]:
def loadData(fn):
    """
    Return preprocessed data
    
    Returns: X and Y where X is pair of sentence (x,x') and y is the label 0 or 1
    """
    data = open(fn)
    data = np.array([example.split("\t") for example in data.readlines()])[1:]
    data_X = []
    data_Y = []
    for d in data:
      data_X += [(d[3], d[4])]
      data_Y += [d[0]]
    return data_X, data_Y


In [0]:
def process(sentence):
  sentence = sentence.lower().split(" ")
  try:
    sentence = sentence[:20]
  except: 
    pass
  return " ".join(sentence)

def preprocessing(data):
    """
    Return preprocessed data

    Args:
        data : sentence pairs
    
    Returns: preprocessed_data
    preprocessed_data : preprocessed dataset 
    """
    preprocessed_data = []
    for p in data:
      preprocessed_data += [(process(p[0]), process(p[1]))]
    return preprocessed_data

### Test train split
Use test train split from sklearn.


In [0]:
def testTrainSplit(data_X,data_Y):
    """
    Return test train data

    Args:
        data_X : sentence pairs
        data_Y: labels
        
    Returns: test train and validation split data 
    """
    x_train, x_valid, y_train, y_valid = train_test_split(data_X, data_Y, test_size=0.1, random_state=0)
    return x_train, x_valid, y_train, y_valid

In [0]:
x_train, y_train = loadData('msr_paraphrase_train.txt')
x_test, y_test = loadData('msr_paraphrase_test.txt')
x_train = preprocessing(x_train)
x_test = preprocessing(x_test)
x_train, x_valid, y_train, y_valid = testTrainSplit(x_train, y_train)

In [0]:
encod = {
    "1": [0, 1],
    "0": [1, 0]
}
y_train_enc = np.array([encod[x] for x in y_train], dtype=np.uint8)
y_valid_enc = np.array([encod[x] for x in y_valid], dtype=np.uint8)
y_test_enc = np.array([encod[x] for x in y_test], dtype=np.uint8)

Implement the step regarding keras Tokenizer in the cell below.<br>
<i> Keep in mind that each example is a pair/tupple of sentence(x,x'), combine them into a single sentence so that your data is a list of sentences before calling fit on text(Tokenizer). There is out of vocabulary option in tokenizer check that out aswell.</i>

In [0]:
# Get vocabular, vocabulary size and numeric word seqeunces for train,test and validation data
tokenizer = Tokenizer(oov_token="<unk>", filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
# tokenizer.word_index['<pad>'] = 0
# tokenizer.index_word[0] = '<pad>'
tmp = [" ".join(x) for x in x_train]
tokenizer.fit_on_texts(tmp)

In [130]:
# def get_vocab(tmp):
#   tmp = [" ".join(x) for x in tmp]
#   tmp = " ".join(tmp).split()
#   vocab = list(set(tmp))
#   return vocab, len(vocab)
# x_train_vocab, x_train_vocab_len = get_vocab(x_train)
# x_valid_vocab, x_valid_vocab_len = get_vocab(x_valid)
# x_test_vocab, x_test_vocab_len = get_vocab(x_test)
vocabulary=list(tokenizer.word_index.keys())#get the vocabular
vocabulary=len(vocabulary)+1#vocabsize
print("Numeric assignments, as you can see zero is reserved for padding\n",tokenizer.word_index,"\n")
word_index = tokenizer.word_index

Numeric assignments, as you can see zero is reserved for padding
 {'<unk>': 1, 'the': 2, 'to': 3, 'of': 4, 'a': 5, 'in': 6, 'and': 7, '\n': 8, 'said': 9, 'on': 10, 'for': 11, 'that': 12, 'was': 13, 'is': 14, 'at': 15, 'with': 16, 'percent': 17, 'he': 18, 'by': 19, 'it': 20, 'from': 21, 'were': 22, 'as': 23, 'be': 24, '1': 25, 'has': 26, 'will': 27, 'or': 28, 'have': 29, 'his': 30, 'an': 31, 'had': 32, 'not': 33, 'are': 34, 'but': 35, 'who': 36, 'year': 37, 'its': 38, 'also': 39, 'they': 40, 'after': 41, 'two': 42, 'new': 43, 'million': 44, 'would': 45, 'this': 46, 'up': 47, '2': 48, 'been': 49, 'u': 50, 'about': 51, 's': 52, 'more': 53, '4': 54, 'one': 55, 'first': 56, 'last': 57, 'we': 58, 'than': 59, 'which': 60, 'their': 61, 'i': 62, 'mr': 63, '0': 64, 'people': 65, 'no': 66, 'shares': 67, 'points': 68, '3': 69, 'cents': 70, 'other': 71, 'company': 72, 'index': 73, 'state': 74, 'police': 75, 'she': 76, '6': 77, 'president': 78, '000': 79, 'her': 80, 'years': 81, '5': 82, 'when': 83,

In [0]:
def transform(tmp):
  train_seqs = tokenizer.texts_to_sequences([tmp])
  padded_seqs = pad_sequences(train_seqs, padding='post', maxlen=20)
  res = padded_seqs[0]
  return res
def transform_data(tmp):
  res = []
  for p in tmp:
    res += [(transform(p[0]), transform(p[1]))]
  return res
x_train_seqs = transform_data(x_train)
x_test_seqs = transform_data(x_test)
x_valid_seqs = transform_data(x_valid)

In [0]:
def separate(tmp):
  a = np.array([x[0] for x in tmp], dtype=np.uint8)
  b = np.array([x[1] for x in tmp])
  return a, b

x_train_seqs_1, x_train_seqs_2 = separate(x_train_seqs)
x_test_seqs_1, x_test_seqs_2 = separate(x_test_seqs)
x_valid_seqs_1, x_valid_seqs_2 = separate(x_valid_seqs)

In [133]:
print(x_train_seqs_2)

[[ 786    4 3522 ... 8967    9    8]
 [3525  955  370 ...    0    0    0]
 [  63 1013 1229 ...  248    8    0]
 ...
 [   9  705   95 ...   59 2121    3]
 [   4 1028   62 ...  148  249   34]
 [3539   67  682 ...    8    0    0]]


Create the model in the cell below:
Try out different sizes for LSTM 50,100,300 and use relu activations. Also report results with Bi-LSTM as well.<br>
<i>To boost performance you can try adding a hidden layer between the lstm and output layer and also by adding a dropout layer in between different layers</i>

In [148]:
#load pretrained embeddings
EMBEDDING_DIM = 300
embeddings_index = {}
with open("glove.6B.300d.txt") as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
print(embedding_matrix.shape)
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
    embedding_matrix[i] = embedding_vector

(12491, 300)


In [187]:
# code model here
inputs1 = Input(shape=(20,))
inputs2 = Input(shape=(20,))
shared_emb = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=20, trainable=False)
emb_1 = shared_emb(inputs1)
emb_2 = shared_emb(inputs2)

shared_lstm = LSTM(300, activation='relu')
lstm_1 = shared_lstm(emb_1)
lstm_2 = shared_lstm(emb_2)

merged = tf.keras.layers.concatenate([lstm_1, lstm_2])
output = Dense(2, activation='softmax')(merged)

model = Model(inputs=[inputs1, inputs2], outputs=output)
model.compile(loss=["categorical_crossentropy"], optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_40 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_41 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, 20, 300)      3747300     input_40[0][0]                   
                                                                 input_41[0][0]                   
__________________________________________________________________________________________________
lstm_15 (LSTM)                  (None, 300)          721200      embedding_17[0][0]         

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
filepath = "setting_" + "model1" + ".hdf5"
logfilepath = "setting_"+"model1" + ".csv"
reduce_lr_rate=0.2
logCallback = CSVLogger(logfilepath, separator=',', append=False)
earlyStopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=0, mode='auto')
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_weights_only=True, verbose=1,
                             save_best_only=True, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr_rate, patience=10,
                              cooldown=0, min_lr=0.0000000001, verbose=0)

callbacks_list = [logCallback, earlyStopping, reduce_lr, checkpoint]


In [0]:
model.fit([x_train_seqs_1, x_train_seqs_2],
          y_train_enc, 
          # callbacks=callbacks_list, 
          validation_data=([x_valid_seqs_1, x_valid_seqs_2], y_valid_enc),
          epochs=50, batch_size=32, verbose=1,shuffle=True)

Epoch 1/50
 86/115 [=====================>........] - ETA: 4s - loss: 0.6802 - accuracy: 0.6733

Use the <b>model.predict</b> method to get predictions. There predictions will be a probability distribution over the lables, to get the desired class take the max value in a prediction vector as the predicted class.<br>

In [0]:
def predict(data, model):
  x_test_seqs = transform_data(data)
  x_test_seqs_1, x_test_seqs_2 = separate(x_test_seqs)
  res = model.predict([x_test_seqs_1, x_test_seqs_2])
  res = np.argmax(res, axis=-1)
  return res

In [185]:
predictions = predict(x_test, model)
labelList=["0", "1"]

from sklearn.metrics import confusion_matrix
test_Y = y_test_enc
test_Y_max=np.argmax(test_Y, axis=-1)
cm=confusion_matrix(test_Y_max,predictions)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm)
cm = pd.DataFrame(cm, labelList,labelList )# matrix,names row,names col,
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(cm, annot=True, annot_kws={"size": 11}, fmt=".2f") # font size
plt.show()

[[0. 1.]
 [0. 1.]]


In [186]:
print("Classification Report\n",classification_report(test_Y_max, predictions, labels=[0,1], target_names = labelList, zero_division=0))

Classification Report
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       578
           1       0.66      1.00      0.80      1147

    accuracy                           0.66      1725
   macro avg       0.33      0.50      0.40      1725
weighted avg       0.44      0.66      0.53      1725



# Task 2 Sentiment Classification
For this task we will be reusing the movie reviews dataset available on <a href="https://www.kaggle.com/c/word2vec-nlp-tutorial/data">kaggle</a> and download the dataset from there. 
We will be using the unlabeledTrainData file and labeledTrainData file. We will use the gensim package to train word2vec embeddings using [gensim](https://radimrehurek.com/gensim/) package and unlabelled train data as in the previous assignmnet. Now instead for creating a single representation for each review we will be using deep learning models for this task. We will use the same archetecture as before but will experiment with different reccurant networks namely RNN, GRU and LSTM.<br> This task might feel like <a href="https://ibb.co/Tgh2XyH">this</a> but since this is a deep learning assignment thus we must use it.
<h3>Data Preperation</h3>
<ul>
    <li> First we need to preprocess the data, convert the data to lower casing(both files). Any other preprocessing procedures are optional but keep in mind that this will affect the performance of your model.</li>
    <li> Split the labeledTrainData data file into test, train and validation in the ratio 20,70,10. Use <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">scikit_test_train_split</a> <br><i><b>Hint:</b> use the splitter twice to get desired data splits.</i></li>
    <li> Next we need the vocabulary, vocabulary size and to convert sentences to numeric sequences by representing each word with a numeric value which will make our implementation easier later on, use <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer">Tokenizer</a> from keras. <br><i>(Fit the tokenizer on train data and use the same tokenizer to convert train,test and validation data to numeric sequences)</i> </li>
    <li>  Use <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences"> pad sequences</a> to add post padding to all sentences that are shorter than maximum sequence length</li>
    <li> Use one hot representation for targets/labels, you can use <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html">scikit learn</a> or <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing">keras preprocessing</a>.</li>
</ul>
<h3>Loading embeddings</h3>
<ul>
    <li> As state before use the gensim package to train the word2vec model on unlabelledTrainData file</li>
    <li> Next we will create a dictionary for our dataset's vocabulary. Copy all the word embeddings for words that are in our vocabulary and in the word2vec model, if a word exists in our vocabulary but does not exist in word2vec model create a zero vector of embedding dimension size and add it to the dictionary.</li>
</ul>
<h3>Create Model</h3>
<ul>
    <li> Here is a visual for the model <img src="sentimentdeep.png">
    <li> Create the model using <a href"https://www.tensorflow.org/guide/keras/functional">functional API</a> or the <a href="https://www.tensorflow.org/api_docs/python/tf/keras/Sequential">Sequential API</a></li>
    <li> Hints: The emebedding layer has a parameter that allows you to use pretrained embeddings</li>



Use can reuse the code snippets from above for call backs, prediction heat map and classification report
<i>You will have provide a label list for this specific dataset inorder for them to run, you are to make the required changes yourself</i>

# We hope all of you are working on your projects and <a href="https://ibb.co/dcpf4vS"> Kudos for completing the assingnment</a>